# 📊 Datathon FIAP - Passos Mágicos
## Modelo Preditivo de Risco de Defasagem

Este notebook desenvolve um modelo de Machine Learning para identificar alunos em risco de defasagem educacional, utilizando os indicadores do PEDE.

**Objetivo:** Criar um modelo preditivo que identifique padrões nos indicadores de desempenho (IDA, IEG, IAA, IPS, IPV, INDE) que permitam prever alunos em risco de defasagem escolar.

**Classificação de Risco (Metodologia Passos Mágicos):**
- **Sem Risco**: Aluno em fase adequada ou adiantado (D ≥ 0)
- **Risco Moderado**: Aluno 1-2 fases atrasado (0 > D ≥ -2)
- **Risco Severo**: Aluno 3+ fases atrasado (D < -2)

**Autor:** Leandro Leme Crespo

---

## 1. Configuração do Ambiente

In [ ]:
# Clonar o repositório do GitHub (executar apenas no Google Colab)
!git clone https://github.com/LeandroCrespo/datathon-passos-magicos.git
print('✅ Repositório clonado com sucesso!')

In [ ]:
# Instalar bibliotecas necessárias
!pip install openpyxl scikit-learn imbalanced-learn -q
print('✅ Bibliotecas instaladas!')

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             precision_score, recall_score, f1_score)
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import pickle

# Configurações
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print('✅ Bibliotecas importadas com sucesso!')

## 2. Carregamento e Preparação dos Dados

In [ ]:
# Carregar dados (do repositório GitHub)
CAMINHO_ARQUIVO = '/content/datathon-passos-magicos/data/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'

# Carregar a primeira planilha (dados de 2024)
df = pd.read_excel(CAMINHO_ARQUIVO)

print(f'📊 Dados carregados: {df.shape[0]:,} alunos, {df.shape[1]} colunas')
print(f'\nColunas disponíveis:')
print(df.columns.tolist())

In [ ]:
# Renomear colunas para padronizar
col_map = {}
for col in df.columns:
    col_lower = col.lower()
    if col_lower == 'iaa': col_map[col] = 'IAA'
    elif col_lower == 'ieg' and 'destaque' not in col_lower: col_map[col] = 'IEG'
    elif col_lower == 'ips': col_map[col] = 'IPS'
    elif col_lower == 'ida' and 'destaque' not in col_lower: col_map[col] = 'IDA'
    elif col_lower == 'ipv' and 'destaque' not in col_lower: col_map[col] = 'IPV'
    elif col_lower == 'ian': col_map[col] = 'IAN'
    elif 'defas' in col_lower: col_map[col] = 'DEFASAGEM'

# Mapear INDE
if 'INDE 22' in df.columns:
    col_map['INDE 22'] = 'INDE'

df = df.rename(columns=col_map)

# Converter para numérico
indicadores = ['IDA', 'IEG', 'IAA', 'IPS', 'IPV', 'IAN', 'INDE', 'DEFASAGEM']
for col in indicadores:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print('✅ Colunas renomeadas e convertidas')
print(f'\nIndicadores disponíveis: {[c for c in indicadores if c in df.columns]}')

## 3. Definição da Variável Alvo (Classificação de Risco)

Seguindo a metodologia oficial da Passos Mágicos:

**D = Fase Efetiva - Fase Ideal**

| Defasagem (D) | Classificação | Classe |
|---------------|---------------|--------|
| D ≥ 0 | Sem Risco (Em fase) | 0 |
| 0 > D ≥ -2 | Risco Moderado | 1 |
| D < -2 | Risco Severo | 2 |

In [ ]:
# Criar variável alvo com 3 classes
def classificar_risco(d):
    """Classifica o risco de defasagem conforme metodologia Passos Mágicos"""
    if pd.isna(d):
        return None
    if d >= 0:
        return 0  # Sem Risco (Em fase ou adiantado)
    elif d >= -2:
        return 1  # Risco Moderado (1-2 fases atrasado)
    else:
        return 2  # Risco Severo (3+ fases atrasado)

df['CLASSE_RISCO'] = df['DEFASAGEM'].apply(classificar_risco)

# Remover registros sem classificação
df = df.dropna(subset=['CLASSE_RISCO'])
df['CLASSE_RISCO'] = df['CLASSE_RISCO'].astype(int)

print('📊 Distribuição da variável alvo (CLASSE_RISCO):')
print(df['CLASSE_RISCO'].value_counts().sort_index())
print(f'\nPercentuais:')
print(f'   Sem Risco (0): {(df["CLASSE_RISCO"] == 0).mean()*100:.1f}%')
print(f'   Risco Moderado (1): {(df["CLASSE_RISCO"] == 1).mean()*100:.1f}%')
print(f'   Risco Severo (2): {(df["CLASSE_RISCO"] == 2).mean()*100:.1f}%')

In [ ]:
# Visualizar distribuição
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
cores = ['#2ecc71', '#f39c12', '#e74c3c']
labels = ['Sem Risco', 'Risco Moderado', 'Risco Severo']
valores = df['CLASSE_RISCO'].value_counts().sort_index()

bars = axes[0].bar(labels, valores.values, color=cores, edgecolor='black')
axes[0].set_title('Distribuição das Classes de Risco', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Quantidade de Alunos')
for bar, val in zip(bars, valores.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                 f'{val}', ha='center', fontsize=12, fontweight='bold')

# Gráfico de pizza
axes[1].pie(valores.values, labels=labels, colors=cores, autopct='%1.1f%%',
            startangle=90, explode=(0, 0.05, 0.1))
axes[1].set_title('Proporção das Classes de Risco', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('distribuicao_risco.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Análise Exploratória dos Indicadores por Classe de Risco

In [ ]:
# Média dos indicadores por classe de risco
# IMPORTANTE: Não usamos IAN pois é derivado da defasagem
indicadores_modelo = ['IDA', 'IEG', 'IAA', 'IPS', 'IPV', 'INDE']
indicadores_existentes = [i for i in indicadores_modelo if i in df.columns]

media_por_classe = df.groupby('CLASSE_RISCO')[indicadores_existentes].mean()
media_por_classe.index = ['Sem Risco', 'Risco Moderado', 'Risco Severo']

print('📊 Média dos Indicadores por Classe de Risco:')
print(media_por_classe.round(2))

In [ ]:
# Visualizar comparação
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(indicadores_existentes))
width = 0.25

bars1 = ax.bar(x - width, media_por_classe.loc['Sem Risco'], width, label='Sem Risco', color='#2ecc71')
bars2 = ax.bar(x, media_por_classe.loc['Risco Moderado'], width, label='Risco Moderado', color='#f39c12')
bars3 = ax.bar(x + width, media_por_classe.loc['Risco Severo'], width, label='Risco Severo', color='#e74c3c')

ax.set_xlabel('Indicador')
ax.set_ylabel('Média')
ax.set_title('Comparação dos Indicadores por Classe de Risco', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(indicadores_existentes)
ax.legend()
ax.set_ylim(0, 10)

plt.tight_layout()
plt.savefig('comparacao_indicadores.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Preparação dos Dados para o Modelo

**Features utilizadas:** IDA, IEG, IAA, IPS, IPV, INDE

**IMPORTANTE:** Não utilizamos o IAN como feature porque ele é derivado diretamente da defasagem (variável alvo). Usar o IAN seria redundante e não permitiria identificar padrões nos outros indicadores.

In [ ]:
# Definir features (SEM IAN)
features = ['IDA', 'IEG', 'IAA', 'IPS', 'IPV', 'INDE']
features_existentes = [f for f in features if f in df.columns]

print(f'📋 Features utilizadas: {features_existentes}')
print(f'\n⚠️ IAN não é utilizado pois é derivado da defasagem (variável alvo)')

# Preparar dados
df_modelo = df[features_existentes + ['CLASSE_RISCO']].dropna()
print(f'\n📊 Registros para modelagem: {len(df_modelo)}')

X = df_modelo[features_existentes]
y = df_modelo['CLASSE_RISCO']

In [ ]:
# Dividir em treino e teste (estratificado)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'📊 Divisão dos dados:')
print(f'   Treino: {len(X_train):,} registros')
print(f'   Teste: {len(X_test):,} registros')
print(f'\n📊 Distribuição no treino:')
print(f'   Sem Risco: {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.1f}%)')
print(f'   Risco Moderado: {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.1f}%)')
print(f'   Risco Severo: {(y_train == 2).sum():,} ({(y_train == 2).mean()*100:.1f}%)')

In [ ]:
# Normalizar features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('✅ Features normalizadas com StandardScaler')

## 6. Balanceamento de Classes com SMOTE

Como temos um desbalanceamento significativo entre as classes (poucos alunos em risco severo), utilizamos a técnica **SMOTE** (Synthetic Minority Over-sampling Technique) para criar exemplos sintéticos das classes minoritárias.

In [ ]:
# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print('📊 Balanceamento com SMOTE:')
print(f'   Antes: {dict(y_train.value_counts().sort_index())}')
print(f'   Depois: {dict(pd.Series(y_train_smote).value_counts().sort_index())}')

## 7. Treinamento do Modelo (Random Forest)

In [ ]:
# Treinar modelo Random Forest
modelo = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    class_weight='balanced'
)

modelo.fit(X_train_smote, y_train_smote)
print('✅ Modelo Random Forest treinado com sucesso!')

## 8. Avaliação do Modelo

In [ ]:
# Fazer predições
y_pred = modelo.predict(X_test_scaled)
y_pred_proba = modelo.predict_proba(X_test_scaled)

# Métricas gerais
print('='*60)
print('📊 RESULTADOS DO MODELO')
print('='*60)
print(f'\n🎯 Acurácia: {accuracy_score(y_test, y_pred)*100:.2f}%')

print('\n📋 Relatório de Classificação:')
print(classification_report(y_test, y_pred, 
                           target_names=['Sem Risco', 'Risco Moderado', 'Risco Severo']))

In [ ]:
# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Sem Risco', 'Risco Moderado', 'Risco Severo'],
            yticklabels=['Sem Risco', 'Risco Moderado', 'Risco Severo'])
ax.set_xlabel('Predito')
ax.set_ylabel('Real')
ax.set_title('Matriz de Confusão', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('matriz_confusao.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n📊 Interpretação da Matriz de Confusão:')
print(f'   Sem Risco corretamente identificados: {cm[0,0]}')
print(f'   Risco Moderado corretamente identificados: {cm[1,1]}')
print(f'   Risco Severo corretamente identificados: {cm[2,2]}')

## 9. Importância das Features

In [ ]:
# Calcular importância das features
importancias = pd.DataFrame({
    'Feature': features_existentes,
    'Importância': modelo.feature_importances_
}).sort_values('Importância', ascending=False)

print('📊 Importância das Features:')
for _, row in importancias.iterrows():
    print(f'   {row["Feature"]}: {row["Importância"]*100:.1f}%')

In [ ]:
# Visualizar importância
fig, ax = plt.subplots(figsize=(10, 6))

cores = plt.cm.Blues(np.linspace(0.4, 0.8, len(importancias)))
bars = ax.barh(importancias['Feature'], importancias['Importância'], color=cores)

ax.set_xlabel('Importância')
ax.set_title('Importância das Features no Modelo', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Adicionar valores nas barras
for bar, val in zip(bars, importancias['Importância']):
    ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
            f'{val*100:.1f}%', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Validação Cruzada

In [ ]:
# Validação cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(modelo, X_train_scaled, y_train, cv=cv, scoring='accuracy')

print('📊 Validação Cruzada (5-fold):')
print(f'   Scores: {scores}')
print(f'   Média: {scores.mean()*100:.2f}%')
print(f'   Desvio Padrão: {scores.std()*100:.2f}%')

## 11. Salvando o Modelo

In [ ]:
# Salvar modelo e artefatos
output_dir = '/content/datathon-passos-magicos/streamlit/'

# Salvar modelo
with open(f'{output_dir}modelo_risco_defasagem.pkl', 'wb') as f:
    pickle.dump(modelo, f)
print(f'✅ Modelo salvo em {output_dir}modelo_risco_defasagem.pkl')

# Salvar scaler
with open(f'{output_dir}scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f'✅ Scaler salvo em {output_dir}scaler.pkl')

# Salvar lista de features
with open(f'{output_dir}features.txt', 'w') as f:
    f.write(','.join(features_existentes))
print(f'✅ Features salvas em {output_dir}features.txt')

# Salvar info do modelo
modelo_info = {
    'features': features_existentes,
    'classes': {0: 'Sem Risco', 1: 'Risco Moderado', 2: 'Risco Severo'},
    'accuracy': accuracy_score(y_test, y_pred),
    'feature_importance': dict(zip(features_existentes, modelo.feature_importances_.tolist()))
}

with open(f'{output_dir}modelo_info.pkl', 'wb') as f:
    pickle.dump(modelo_info, f)
print(f'✅ Info do modelo salva em {output_dir}modelo_info.pkl')

## 12. Exemplo de Predição

In [ ]:
# Exemplo de predição para cada classe
print('='*60)
print('📊 EXEMPLOS DE PREDIÇÃO')
print('='*60)

for classe in [0, 1, 2]:
    exemplo = df_modelo[df_modelo['CLASSE_RISCO'] == classe].iloc[0]
    X_exemplo = scaler.transform([exemplo[features_existentes].values])
    pred = modelo.predict(X_exemplo)[0]
    proba = modelo.predict_proba(X_exemplo)[0]
    
    classe_nome = ['Sem Risco', 'Risco Moderado', 'Risco Severo'][classe]
    pred_nome = ['Sem Risco', 'Risco Moderado', 'Risco Severo'][pred]
    
    print(f'\n🎯 Exemplo de aluno "{classe_nome}":')
    print(f'   Indicadores: {dict(exemplo[features_existentes].round(2))}')
    print(f'   Predição: {pred_nome}')
    print(f'   Probabilidades: Sem Risco={proba[0]:.2f}, Moderado={proba[1]:.2f}, Severo={proba[2]:.2f}')

## 13. Conclusões

### Resultados do Modelo

O modelo Random Forest foi treinado para classificar alunos em três categorias de risco de defasagem:

| Métrica | Valor |
|---------|-------|
| **Acurácia Geral** | ~75% |
| **Recall (Sem Risco)** | ~69% |
| **Recall (Risco Moderado)** | ~82% |
| **Recall (Risco Severo)** | Baixo (poucos exemplos) |

### Importância das Features

Os indicadores mais importantes para prever o risco de defasagem são:

1. **INDE** (~35%) - Índice geral de desenvolvimento
2. **IPV** (~16%) - Ponto de Virada
3. **IDA** (~15%) - Desempenho Acadêmico
4. **IAA** (~13%) - Autoavaliação
5. **IEG** (~12%) - Engajamento
6. **IPS** (~10%) - Psicossocial

### Utilidade Prática

O modelo permite:
- Identificar **padrões nos indicadores** associados a alunos em defasagem
- Alertar sobre alunos que **hoje estão bem**, mas têm padrões similares aos que estão em defasagem
- Permitir **intervenção preventiva** antes que a defasagem aconteça

### Limitações

- Poucos exemplos de Risco Severo (3.3% dos dados)
- Modelo treinado apenas com dados de 2024
- Recomenda-se usar como apoio à decisão, não como única fonte